In [ ]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate()

  Activating environment at `~/tinympc-julia/state_constraints/Project.toml`


In [26]:
using RobotZoo:Quadrotor
using RobotDynamics
using ForwardDiff
using TrajOptPlots
using BlockDiagonals
using LinearAlgebra
using StaticArrays
using SparseArrays
using ControlSystems

using MeshCat
using ColorTypes
using GeometryBasics: HyperSphere, HyperRectangle, Cylinder, Vec, Point, Mesh
using CoordinateTransformations
using Rotations

using Plots
using Printf

include("../quaternion-stuff.jl")

E (generic function with 1 method)

In [27]:
function export_mat_to_c(declare, data)
    str = "tinytype" * declare * "= {\n"
    for i = 1:size(data, 1)
        str = str * "  "
        for j = 1:size(data, 2)
            if i == size(data, 1) && j == size(data, 2)
                this_str = @sprintf("%.7f\t", data[i, j])
            else
                this_str = @sprintf("%.7f,\t", data[i, j])
            end
            str = str * this_str
        end
        str = str * "\n"
    end
    str = str * "};"
    return str
end

function export_diag_to_c(declare, data)
    str = "tinytype" * declare * "= {"
    for i = 1:size(data, 1)
        if i == size(data, 1)
            this_str = @sprintf("%.7f", data[i, i])
        else
            this_str = @sprintf("%.7f,\t", data[i, i])
        end
        str = str * this_str
    end
    str = str * "};"
    return str
end

function export_Xref_to_c(declare, data)
    str = "tinytype" * declare * "= {\n"
    for i = 1:size(data, 1)
        str = str * "  "
        for j = 1:nx
            if i == size(data, 1) && j == nx
                this_str = @sprintf("%.7f\t", data[i][j])
            else
                this_str = @sprintf("%.7f,\t", data[i][j])
            end
            # str = str * this_str * "f"
            str = str * this_str
        end
        str = str * "\n"
    end
    str = str * "};"
    return str
end


export_Xref_to_c (generic function with 1 method)

In [28]:
# Quadrotor parameters
m = 0.035
ℓ = 0.046/sqrt(2)
J = [16.6e-6 0.83e-6 0.72e-6;
    0.83e-6 16.6e-6 1.8e-6;
    0.72e-6 1.8e-6 29.3e-6];
gravity = 9.81
thrustToTorque = 0.0008
scale = 65535
kt = 2.245365e-6*scale # u is PWM in range [0...1]
km = kt*thrustToTorque #4.4733e-8

# From "design of a trajectory tracking controller for a nanoquadcopter"
# J = [1.395e-5 0 0;
#     0 1.436e-5 0;
#     0 0 2.173e-5];
# kt = .2025
# km = .11

# h = 1/8000
# h = 1/5000
# h = 1/1000
# h = 1/500
# h = 1/250
# h = 1/100
h = 1/50
# h = 1/25
# h = 1/10

Nx = 13     # number of states (quaternion)
Nx̃ = 12     # number of states (linearized)
Nu = 4      # number of controls
nx = Nx̃
nu = Nu

#Goal state
uhover = (m*gravity/kt/4)*ones(4) # m = 30g and max thrust = 60g
# uhover = [0.701, 0.701, 0.832, 0.732]
rg = [0.0; 0; 0.0]
qg = [1.0; 0; 0; 0]
vg = zeros(3)
ωg = zeros(3)
xg = [rg; qg; vg; ωg];

In [29]:
uhover

4-element Vector{Float64}:
 0.5833333520642209
 0.5833333520642209
 0.5833333520642209
 0.5833333520642209

In [30]:
function quad_dynamics(x,u)
  r = x[1:3]
  q = x[4:7]/norm(x[4:7]) #normalize q just to be careful
  v = x[8:10]
  ω = x[11:13]
  Q = qtoQ(q)
  
  ṙ = v
  q̇ = 0.5*L(q)*H*ω
  
  b = 1/m

  v̇ = [0; 0; -gravity] + (1/m)*Q*[zeros(2,4); kt*ones(1,4)]*u 

  ω̇ = J\(-hat(ω)*J*ω + [-ℓ*kt -ℓ*kt ℓ*kt ℓ*kt; -ℓ*kt ℓ*kt ℓ*kt -ℓ*kt; -km km -km km]*u)
  
  return [ṙ; q̇; v̇; ω̇]
end
function quad_dynamics_rk4(x,u)
  #RK4 integration with zero-order hold on u
  f1 = quad_dynamics(x, u)
  f2 = quad_dynamics(x + 0.5*h*f1, u)
  f3 = quad_dynamics(x + 0.5*h*f2, u)
  f4 = quad_dynamics(x + h*f3, u)
  xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
  xn[4:7] = xn[4:7]/norm(xn[4:7]) #re-normalize quaternion
  return xn
end

quad_dynamics_rk4 (generic function with 1 method)

In [31]:
#Linearize dynamics about hover
Adyn = ForwardDiff.jacobian(x->quad_dynamics_rk4(x,uhover),xg)
Bdyn = ForwardDiff.jacobian(u->quad_dynamics_rk4(xg,u),uhover);
Ãdyn = Array(E(qg)'*Adyn*E(qg))
B̃dyn = Array(E(qg)'*Bdyn);

In [32]:
display(Ãdyn)
display(B̃dyn)
display(cond(Ãdyn))
display(cond(B̃dyn))

12×12 Matrix{Float64}:
 1.0  0.0  0.0   0.0          0.003924     …   1.308e-5     -2.96444e-27
 0.0  1.0  0.0  -0.003924     0.0              0.0          -8.30044e-27
 0.0  0.0  1.0   5.92889e-25  1.66009e-24      8.30044e-27   0.0
 0.0  0.0  0.0   1.0          1.19137e-20     -1.91331e-23   2.82765e-23
 0.0  0.0  0.0  -1.19137e-20  1.0              0.01         -1.14919e-23
 0.0  0.0  0.0  -1.26918e-21  4.53279e-22  …   4.53279e-24   0.01
 0.0  0.0  0.0   0.0          0.3924           0.001962     -1.57202e-24
 0.0  0.0  0.0  -0.3924       0.0              1.35643e-23  -4.01899e-24
 0.0  0.0  0.0   1.18578e-22  3.32018e-22      2.49013e-24   0.0
 0.0  0.0  0.0   0.0          0.0             -1.76536e-20   7.21376e-21
 0.0  0.0  0.0   0.0          0.0          …   1.0          -2.99429e-21
 0.0  0.0  0.0   0.0          0.0              9.06558e-22   1.0

12×4 Matrix{Float64}:
 -1.80966e-5    1.98998e-5    1.8153e-5    -1.99562e-5
  1.80082e-5    1.98315e-5   -1.80283e-5   -1.98114e-5
  0.000840857   0.000840857   0.000840857   0.000840857
 -0.0275355    -0.0303234     0.0275663     0.0302926
 -0.0276707     0.0304278     0.027757     -0.0305141
  0.00197477   -0.000722364  -0.00278438    0.00153197
 -0.00361933    0.00397996    0.00363061   -0.00399124
  0.00360164    0.0039663    -0.00360567   -0.00396227
  0.0840857     0.0840857     0.0840857     0.0840857
 -5.50709      -6.06468       5.51325       6.05852
 -5.53414       6.08557       5.55139      -6.10282
  0.394954     -0.144473     -0.556875      0.306394

1.4778280166701019

75.83971616632428

In [57]:
vis = Visualizer()
quad_model = Quadrotor()
TrajOptPlots.set_mesh!(vis, quad_model)

function TransformPlane(visObject, center, dir)

    tol = 1e-4
    x_axis = [1.0; 0.0; 0.0]

    # Compute axis angle
    dot = x_axis'*dir
    if dot > 1-tol
        axis = x_axis
    elseif dot < -1+tol
        axis = -x_axis
    else
        axis = cross(x_axis, dir)
    end
    angle = acos(x_axis'*dir)

    settransform!(visObject, Translation(center...) ∘ LinearMap(AngleAxis(angle, axis...)))
end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8702
└ @ MeshCat /home/tinympc/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


TransformPlane (generic function with 1 method)

In [107]:
# Cost weights
max_dev_x = [0.01; 0.01; 0.01; 0.5; 0.5; 0.05;  0.5; 0.5; 0.5;  0.7; 0.7; 0.5]
max_dev_u = [0.1; 0.1; 0.1; 0.1]
# max_dev_u = [0.02; 0.02; 0.02; 0.02]
Q = diagm(1 ./(max_dev_x.^2))
R = diagm(1 ./(max_dev_u.^2))
# Q = spdiagm([100, 100, 100,  4, 4, 400,  4, 4, 4,  2, 2, 4])
# R = spdiagm([4, 4, 4, 4])
# Q = spdiagm([100, 100, 400,  4, 4, 100,  4, 4, 4,  2, 2, 4]) #.*5^2
# R = spdiagm([5, 5, 5, 5]) #.*5^2
Qf = Q
# ρ = 5
# R̃ = R + ρ*I;
# Q̃ = Q + ρ*I;
# Q̃f = Qf + ρ*I;
;

In [116]:
R

4×4 Matrix{Float64}:
 100.0    0.0    0.0    0.0
   0.0  100.0    0.0    0.0
   0.0    0.0  100.0    0.0
   0.0    0.0    0.0  100.0

In [103]:
# Adync = ForwardDiff.jacobian(x->quad_dynamics(x,uhover),xg)
# Bdync = ForwardDiff.jacobian(u->quad_dynamics(xg,u),uhover);
# Ãdync = Array(E(qg)'*Adync*E(qg))
# B̃dync = Array(E(qg)'*Bdync);

# Kc = lqr(Continuous, Ãdync, B̃dync, Q̃, R̃)
# display(Kc)

In [108]:
# Number of backward pass related matrix sets to cache
ρ_list_length = 1

# Allocate cache
cache = (
    A = Ãdyn,
    B = B̃dyn,
    # ρ_list = [diagm(ones(nu)) for _ in 1:ρ_list_length],
    ρ_list = [zeros(1) for _ in 1:ρ_list_length],
    Kinf_list = [zeros(nu,nx) for _ in 1:ρ_list_length],
    Pinf_list = [zeros(nx,nx) for _ in 1:ρ_list_length],
    Quu_inv_list = [zeros(nu,nu) for _ in 1:ρ_list_length],
    AmBKt_list = [zeros(nx,nx) for _ in 1:ρ_list_length],
    coeff_d2p_list = [zeros(nx,nu) for _ in 1:ρ_list_length],
)

# Precompute all cached matrices for multiple ρ values starting at ρ = .1 and multiplying by 5
for k = 1:ρ_list_length
    # ρ
    # ρ = diagm([1; 1; 1; 1])*1*5^(k-1)
    ρ = 5*(5^(k-1));
    R̃ = R + ρ*I;
    Q̃ = Q + ρ*I;
    Q̃f = Qf + ρ*I;

    # K1 = dlqr(cache.A, cache.B, Q̃, R̃)
    # display(K)
    
    K = zeros(nu,nx)
    P = zeros(nx,nx)
    Kprev = zeros(nu,nx)
    Pprev = zeros(nx,nx)
    
    # Compute Kinf, Pinf
    riccati_iters = 0
    riccati_err = 1e-10
    Pprev = Q̃f
    while true
        K = (R̃ + cache.B'*Pprev*cache.B)\(cache.B'*Pprev*cache.A);
        P = Q̃ + cache.A'*Pprev*(cache.A - cache.B*K);
        if maximum(abs.(K - Kprev)) < riccati_err
            break
        end
        Kprev = K
        Pprev = P
        riccati_iters += 1
    end
    display(K)
    # display(maximum(K1-K))

    # Cache precomputed values
    cache.ρ_list[k] .= ρ;
    # cache.Kinf_list[k] .= K[1];
    # cache.Pinf_list[k] .= P[1];
    cache.Kinf_list[k] .= K;
    cache.Pinf_list[k] .= P;
    cache.Quu_inv_list[k] .= (R̃ + cache.B'*cache.Pinf_list[k]*cache.B)\I;
    cache.AmBKt_list[k] .= (cache.A - cache.B*cache.Kinf_list[k])';
    cache.coeff_d2p_list[k] .= cache.Kinf_list[k]'*R̃ - cache.AmBKt_list[k]*cache.Pinf_list[k]*cache.B;
end

4×12 Matrix{Float64}:
 -1.33252   1.33521  4.28413  -1.23061  …  -0.0546082  -0.0647876  -0.279677
  1.23343   1.22886  4.28413  -1.05931     -0.0369556   0.0609616   0.279334
  1.31463  -1.32819  4.28413   1.15196      0.040783    0.0302132  -0.278478
 -1.21554  -1.23588  4.28413   1.13796      0.0507808  -0.0263872   0.278821

In [105]:
# log.(eigvals(Ãdyn-B̃dyn*cache.Kinf_list[1]))
abs.(eigvals(Ãdyn-B̃dyn*cache.Kinf_list[1]))

12-element Vector{Float64}:
 0.08449506260033615
 0.0987269619420087
 0.9454297777422224
 0.9454400190301139
 0.9533006926901975
 0.9533006926901975
 0.9548608956785447
 0.9548608956785447
 0.9568599834716656
 0.9568599834716656
 0.9569469053227603
 0.9569469053227603

In [109]:
boilerplate = "#pragma once\n\n#include <tinympc/types.hpp>\n\n"

rho_string = "tinytype rho_value = " * string(cache.ρ_list[1][1]) * ";\n\n"

A_data_string = export_mat_to_c(" Adyn_data[NSTATES*NSTATES] ", cache.A) * "\n\n"
B_data_string = export_mat_to_c(" Bdyn_data[NSTATES*NINPUTS] ", cache.B) * "\n\n"

Kinf_data_string = export_mat_to_c(" Kinf_data[NINPUTS*NSTATES] ", cache.Kinf_list[1]) * "\n\n"
Pinf_data_string = export_mat_to_c(" Pinf_data[NSTATES*NSTATES] ", cache.Pinf_list[1]) * "\n\n"
Quu_inv_data_string = export_mat_to_c(" Quu_inv_data[NINPUTS*NINPUTS] ", cache.Quu_inv_list[1]) * "\n\n"
AmBKt_data_string = export_mat_to_c(" AmBKt_data[NSTATES*NSTATES] ", cache.AmBKt_list[1]) * "\n\n"
coeff_d2p_data_string = export_mat_to_c(" coeff_d2p_data[NSTATES*NINPUTS] ", cache.coeff_d2p_list[1]) * "\n\n"

Q_data_string = export_diag_to_c(" Q_data[NSTATES]", Q) * "\n\n"
Qf_data_string = export_diag_to_c(" Qf_data[NSTATES]", Qf) * "\n\n"
R_data_string = export_diag_to_c(" R_data[NINPUTS]", R) * "\n\n"

open("quadrotor_50hz_params_3.hpp", "w") do file
    write(file, boilerplate);
    write(file, rho_string);
    write(file, A_data_string);
    write(file, B_data_string);
    write(file, Kinf_data_string);
    write(file, Pinf_data_string);
    write(file, Quu_inv_data_string);
    write(file, AmBKt_data_string);
    write(file, coeff_d2p_data_string);
    write(file, Q_data_string);
    write(file, Qf_data_string);
    write(file, R_data_string);
end

81

In [131]:
# Create reference sketch trajectory
Tfinal = 5
N = Int(Tfinal/h)+1

X̃ref = [zeros(nx) for i = 1:N]
# Xref = [[0; 0; 1; 1; zeros(9)] for i = 1:N] # Point
Xref = [[0; t; 1; 1; zeros(9)] for t in range(-1.5, 1.5, length=N)] # Line
# Xref = [[sin(t); cos(t); 1; 1; zeros(9)] for t in range(-pi/2, 3*pi/2, length=N)] # Circle
# Xref = [[sin(2*t); 2*cos(t); 1; 1; zeros(9)] for t in range(-pi/2, 3*pi/2, length=N)] # Figure-eight
Uref = [zeros(nu) for i = 1:N-1]

# Compute reference velocity from reference position
for i = 1:N-1
    Xref[i][8:10] = (Xref[i+1][1:3] - Xref[i][1:3])/h
end

# Convert (13 state) Xref to reduced form (12 state) X̃ref
for k = 1:N
    x = Xref[k]
    qx = x[4:7]
    ϕ = qtorp(L(qg)'*qx)   
    X̃ref[k] = [x[1:3]; ϕ; x[8:10]; x[11:13]]
end

# Set initial state
x0 = X̃ref[1] + [0; 0; 0; zeros(9)]
# x0 = [0.1; 0.1; 0; zeros(9)]

# Visualize reference
delete!(vis["XrefLine"])
XrefLine = [Point(x_[1], x_[2], x_[3]) for x_ in Xref]
setobject!(vis["XrefLine"], Object(PointCloud(XrefLine), 
        LineBasicMaterial(color=Colors.RGBA(0.,0.45,1.)), "Line"))
;

In [133]:
X̃_ref_string = export_Xref_to_c(" Xref_data[NTOTAL*NSTATES] ", X̃ref)

open("quadrotor_50hz_line_5s.hpp", "w") do file
    write(file, boilerplate)
    write(file, X̃_ref_string)
end

34050

In [126]:
include("../tinyMPC-ADMM-dt-state.jl")

speye(N) = spdiagm(ones(N))


# Clean up MeshCat environment from previous run
for k in 1:10000
    delete!(vis["xlin " * string(k)])
    delete!(vis["linearized constraint " * string(k)])
    delete!(vis["obstacle " * string(k)])
    delete!(vis["bounding obstacle " * string(k)])
end



# Used for skipping skip_obs number of linearized obstacle constraints (skips both visualization and computation)
skip_obs = 1

# MPC setup
Nh = 25


# Create obstacle in MeshCat
obstacle_center = [0.0; 2.0; 0.5]
obstacle_velocity = 1 * h

r_vis = 0.1
r_obstacle = 0.75

# Create visualization data to populate during MPC solution

# bounding_material = MeshPhongMaterial(color=RGBA(1, 1, 1, 0.25))
# thin_rect_material = MeshPhongMaterial(color=RGBA(0, 1, 0, 0.25))
# constraintPlanes = [setobject!(vis["linearized constraint " * string(k)], HyperRectangle(Vec(-0.0005,-0.5,-0.5), Vec(0.001,1,1)), thin_rect_material) for k in 1:skip_obs:Nh]


# Instantiate TinyMPC-ADMM variables

x = [zeros(nx) for i = 1:Nh]
u = [zeros(nu) for i = 1:Nh-1]

q = [zeros(nx) for i = 1:Nh];
r = [zeros(nu) for i = 1:Nh-1];

p = [zeros(nx) for i = 1:Nh];      # cost to go linear term
p[Nh] = q[Nh];

d = [zeros(nu) for i = 1:Nh-1];    # feedforward control

# Input constraints
umin = -[1.0; 1.0; 1.0; 1.0]*.5
umax = [1.0; 1.0; 1.0; 1.0]*.5

# State constraints
# xmin = [-Inf .* ones(nx) for i = 1:Nh]
# xmax = [[Inf; 0.5; Inf .* ones(10)] for i = 1:Nh]
xmin = [-Inf for i = 1:Nh] # Only one -Inf per knot point because only one inequality constraint (for now)
# xmax = [zeros(1) for i = 1:Nh]
xmax = [ones(1)*Inf for i = 1:Nh]

# Aineq = [speye(nx) for i = 1:Nh]
Aineq = [zeros(nx) for i = 1:Nh]

# for k = 1:skip_obs:Nh
#     xc = obstacle_center - X̃ref[k][1:3]
#     a = xc/norm(xc)

#     Aineq[k][1:3] = a'

#     q_c = obstacle_center - r_obstacle*a
#     b = a'*q_c
#     xmax[k][1] = b
#     TransformPlane(constraintPlanes[cld(k,skip_obs)], q_c, a)
# end



# Simulate
Nsim = N - Nh
xhist = [zeros(nx+1) for _ = 1:Nsim] # nx+1 because not reduced form
xhist[1] = [x0[1:3]; rptoq(x0[4:6]); x0[7:12]]
uhist = [zeros(nu) for _ = 1:Nsim-1]
@time for k = 1:Nsim-1 # Nsim-1 knotpoints for control, Nsim knotpoints for state

    # State slack (auxiliary) and dual variables
    v = [zeros(nx) for i = 1:Nh]
    vnew = [zeros(nx) for i = 1:Nh]
    g = [zeros(nx) for i = 1:Nh]

    # Input slack (auxiliary) and dual variables
    z = [zeros(nu) for i = 1:Nh-1]
    znew = [zeros(nu) for i = 1:Nh-1]
    y = [zeros(nu) for i = 1:Nh-1]

    x[1] = [xhist[k][1:3]; qtorp(xhist[k][4:7]); xhist[k][8:end]]

    # params.constraint_A .= Aineq
    # params.umin .= umin
    # params.umax .= umax
    # params.xmin .= xmin
    # params.xmax .= xmax
    # params.Xref .= X̃ref[k:k+Nh-1]
    # params.Uref .= Uref[k:k+Nh-2]
    
    params = (
        N = Nh,
        Q = Q, R = R, Qf = Qf,
        constraint_A = Aineq,
        umin = umin, umax = umax,
        xmin = xmin, xmax = xmax,
        Xref = X̃ref[k:k+Nh-1], Uref = Uref[k:k+Nh-2], 
        cache = cache,

        ρ_index = [1]
    )

    # Solve
    Δu, status, iter = solve_admm!(vis, params, q, r, p, d, x,v,vnew,g, u,z,znew,y; abs_tol=1e-3, max_iter=20)
    # @show iter, status
    # display("iter: " * string(iter))
    # display("h[z]: " * string(x[Nh][3]))
    # display("h: " * string(x[Nh]))
    
    # Roll out x_k+1 with actual dynamics
    uhist[k] = Δu[1] + uhover
    xhist[k+1] = quad_dynamics_rk4(xhist[k], uhist[k])
    
    # Roll out with real dynamics
    x_real = [zeros(nx+1) for _ = 1:Nh]
    x_real[1] = xhist[k]
    for j = 1:Nh-1
        x_real[j+1] = quad_dynamics_rk4(x_real[j], uhist[k])
    end

    # Visualize solution x
    x_linearized = [Point(xp[1], xp[2], xp[3]) for xp in x_real]
    if k > 1
        x_lin_name_prev = "xlin " * string(k-1)
        setvisible!(vis[x_lin_name_prev], false)
    end
    x_lin_name = "xlin " * string(k)
    setobject!(vis[x_lin_name], Object(PointCloud(x_linearized), 
        LineBasicMaterial(color=Colors.RGBA(1,0.6,0.0)), "Line"))

    # Create constraints for each knot point in the horizon
    #   assuming the obstacle's center moves at constant velocity between each step
    obstacle_to_quad = x_real[1][1:3] - obstacle_center
    obstacle_to_quad_dir = obstacle_to_quad / norm(obstacle_to_quad)
    obstacle_dist_traveled = obstacle_velocity * obstacle_to_quad_dir
    
    # # Update linearized obstacle constraints
    # if k > 1
    #     for i = 1:Nh
    #         setvisible!(vis["linearized constraint " * string((k-2)*Nh + i)], false)
    #     end
    # end
    # constraintPlanes = [setobject!(vis["linearized constraint " * string((k-1)*Nh + i)], HyperRectangle(Vec(-0.0005,-0.5,-0.5), Vec(0.001,1,1)), thin_rect_material) for i in 1:skip_obs:Nh]
    for j = 1:skip_obs:Nh
        obstacle_center_prediction = obstacle_center + obstacle_dist_traveled*j
        xc = obstacle_center_prediction - x_real[j][1:3]
        a = xc/norm(xc) # hyperplane projection expects normalized a

        Aineq[j][1:3] = a'

        q_c = obstacle_center_prediction - r_obstacle*a
        b = a'*q_c
        xmax[j][1] = b
        TransformPlane(constraintPlanes[cld(j,skip_obs)], q_c, a)
    end

    # Update actual obstacle location at x=0
    # obstacle_center += obstacle_dist_traveled
    
    # # Visualize cylinder
    # if k > 1
    #     setvisible!(vis["obstacle " * string(k-1)], false)
    #     setvisible!(vis["bounding obstacle " * string(k-1)], false)
    # end
    # cyl_vis = setobject!(vis["obstacle " * string(k)], Cylinder(Point(0, 0, -0.5), Point(0, 0, 0.5), r_vis))
    # cyl_avoid = setobject!(vis["bounding obstacle " * string(k)], Cylinder(Point(0, 0, -5.05), Point(0, 0, 5.05), r_obstacle), bounding_material)
    # settransform!(cyl_vis, Translation(obstacle_center[1], obstacle_center[2], obstacle_center[3]))

    # # Visualize sphere
    # if k > 1
    #     setvisible!(vis["obstacle " * string(k-1)], false)
    #     setvisible!(vis["bounding obstacle " * string(k-1)], false)
    # end
    # setobject!(vis["obstacle " * string(k)], HyperSphere(Point(obstacle_center...), r_vis))
    # setobject!(vis["bounding obstacle " * string(k)], HyperSphere(Point(obstacle_center...), r_obstacle), bounding_material)
end

delete!(vis["xHistline"])
xHistline = [Point(x_[1], x_[2], x_[3]) for x_ in xhist]
setobject!(vis["xHistline"], Object(PointCloud(xHistline), 
LineBasicMaterial(color=Colors.RGBA(0.75,0.5,0.8)), "Line"))

  1.669201 seconds (5.33 M allocations: 475.149 MiB, 9.09% gc time, 52.22% compilation time)


MeshCat Visualizer with path /meshcat/xHistline at http://127.0.0.1:8702

In [127]:
X1 = [SVector{13}(x_) for x_ in xhist];
visualize!(vis, quad_model, Tfinal, X1)

In [35]:
# constraintPlanes = [setobject!(vis["linearized constraint " * string(k)], HyperRectangle(Vec(0.0,0.0,0.0), Vec(0.001,2.0,1.0)), thin_rect_material) for k in 1:Nh]
for k in 1:N
    delete!(vis["linearized constraint " * string(k)])
end

# Create MeshCat animation
anim = MeshCat.Animation()

for k = 1:Nsim-1
    atframe(anim, k-1) do
        # Set visibility of each object to false
        if k == 1
            for j = 1:Nsim
                for i = 1:Nh
                    setvisible!(vis["linearized constraint " * string((j-1)*Nh + i)], false)
                end
                setvisible!(vis["xlin " * string(j)], false)
                setvisible!(vis["obstacle " * string(j)], false)
                setvisible!(vis["bounding obstacle " * string(j)], false)
            end
        end

        # Change visible obstacle
        setvisible!(vis["obstacle " * string(k)], true)
        setvisible!(vis["bounding obstacle " * string(k)], true)
        if k > 1
            setvisible!(vis["obstacle " * string(k-1)], false)
            setvisible!(vis["bounding obstacle " * string(k-1)], false)
        end


        # Update obstacle constraint visuals
        for i = 1:Nh
            setvisible!(vis["linearized constraint " * string((k-1)*Nh + i)], true)
        end
        if k > 1
            for i = 1:Nh
                setvisible!(vis["linearized constraint " * string((k-2)*Nh + i)], false)
            end
        end
        

        # Change visible horizon
        setvisible!(vis["xlin " * string(k)], true)
        if k > 1
            setvisible!(vis["xlin " * string(k-1)], false)
        end

        # Update quadrotor transform
        xbar = RBState(quad_model, xhist[k])
        if quad_model.ned
            rq = position(xbar)
            vq = linear_velocity(xbar)
            rq = SA[rq[1],-rq[2],-rq[3]]
            vq = SA[vq[1],-vq[2],-vq[3]]
            xbar = RBState(rq, RotX(pi)*orientation(xbar), vq, angular_velocity(xbar)) 
        end
        settransform!(vis["robot"], Translation(position(xbar)) ∘ LinearMap(UnitQuaternion(orientation(xbar))))
    end
end

setanimation!(vis, anim)

In [36]:
MeshCat.convert_frames_to_video(
    "/Users/anoushkaalavill/Documents/REx_Lab/tinympc-julia/state_constraints/state_constraints_tinympc.tar")

ErrorException: The output path /Users/anoushkaalavill/Documents/REx_Lab/tinympc-julia/state_constraints/output.mp4 already exists. To overwrite that file, you can pass `overwrite=true` to this function

In [37]:
a = [2.0; 3.0; 5.0]
a = a/norm(a)

3-element Vector{Float64}:
 0.3244428422615251
 0.48666426339228763
 0.8111071056538127

In [38]:
include("../tinyMPC-ADMM-dt-state.jl")

speye(N) = spdiagm(ones(N))

params = (
    N = Nh,
    Q = Q, R = R, Qf = Qf,
    constraint_A = Aineq,
    umin = umin, umax = umax,
    xmin = xmin, xmax = xmax,
    Xref = X̃ref[1:Nh], Uref = Uref[1:Nh-1], 
    cache = cache,

    ρ_index = [1]
)

# MPC setup
Nh = 20

# Instantiate TinyMPC-ADMM variables
x = [zeros(nx) for i = 1:Nh]
u = [zeros(nu) for i = 1:Nh-1]

q = [zeros(nx) for i = 1:Nh];
r = [zeros(nu) for i = 1:Nh-1];

p = [zeros(nx) for i = 1:Nh];      # cost to go linear term
p[Nh] = q[Nh];

d = [zeros(nu) for i = 1:Nh-1];    # feedforward control

# Input constraints
umin = -[1.0; 1.0; 1.0; 1.0]*.5
umax = [1.0; 1.0; 1.0; 1.0]*.5

# State constraints
xmin = [-Inf for i = 1:Nh] # Only one -Inf per knot point because only one inequality constraint (for now)
xmax = [zeros(1) for i = 1:Nh]

# Aineq = [speye(nx) for i = 1:Nh]
Aineq = [zeros(nx) for i = 1:Nh]
# Aineq[1] = [0.32444; 0.48666; 0.81111; zeros(9)]

# xmax[1] = [1]

# State slack (auxiliary) and dual variables
v = [zeros(nx) for i = 1:Nh]
vnew = [zeros(nx) for i = 1:Nh]
g = [zeros(nx) for i = 1:Nh]

# Input slack (auxiliary) and dual variables
z = [zeros(nu) for i = 1:Nh-1]
znew = [zeros(nu) for i = 1:Nh-1]
y = [zeros(nu) for i = 1:Nh-1]

x[1] = params.Xref[1]

# display(x)
Δu, status, iter = solve_admm!(vis, params, q, r, p, d, x,v,vnew,g, u,z,znew,y; abs_tol=1e-3, max_iter=100)
@show iter

Δu, status, iter = solve_admm!(vis, params, q, r, p, d, x,v,vnew,g, u,z,znew,y; abs_tol=1e-3, max_iter=100)
@show iter

# p[params.N] = -params.Qf*params.Xref[params.N]

# for k = 1:(params.N-1)
#     z[k] .= min.(params.umax, max.(params.umin, u[k] + y[k]))
# end
# for k = 1:params.N
#     v[k] .= project_hyperplane(k, vis, x[k] + g[k], params.constraint_A[k], params.xmax[k][1])
# end
# update_linear_cost!(v,g,z,y,p,q,r,params)

# backward_pass_grad!(q, r, p, d, params)
# forward_pass!(d, x, u, params)
# display(params.cache.Quu_inv_list[1])
# display(params.cache.B)
# display(params.cache.AmBKt_list[1])
# display(params.cache.Kinf_list[1])
# display(params.cache.coeff_d2p_list[1])


BoundsError: BoundsError: attempt to access 1-element Vector{Int64} at index [2]